In [5]:
# Begin query
import json
from pprint import pprint # Used for debugging

# Creating memory allocation to PSC Index json file
with open("Data/JSON/PCS_Tables.json") as tables_file:
    PCS_tables = json.load(tables_file)
    
# Aggregating tables into one list
tables = [table for table in PCS_tables["ICD10PCS.tabular"]["pcsTable"]]

In [24]:
class PCS_Table:
    def __init__(self):
        pass
    
    def find_table_from_string(self, tab_codes: str) -> dict:
        # This method returns an object containing a PCS table whose pos 1,2,3 codes match what is passed as arguement
        # This method only works when pos 1,2,3 codes are provided in tab_codes in s
        found = False # Flag used to determine if table found
        # Ensure all individual codes inside tab_codes are capitalized
        tab_codes = tab_codes.upper()
        for table in tables:
            # Extract pos 1,2,3 codes and join codes together
            code = "".join((tab_code["label"][0]["_code"] for tab_code in table["axis"]))
            # Find matching table object
            if code == tab_codes:
                found = True
                return table
        # If did not find a match, raise error
        if not found:
            raise LookupError(f"Did not find matching table to '{tab_codes}'. Must include code values for pos 1,2,3 of the PCS table you are looking for. Be sure to not mistake a '0', for 'O'.")
    
    def search_pcs_row_for_components(self, table: dict or str, query_text: str, category=None) -> dict:
        # Search pos 4,5,6,7 to find match to index text
        # This function finds only the first pcs component and returns its code
        # Get table object if not passed to function as argument
        # If argument 'category' passed...
        if category:
            # Analyze 'table' agruement to convert to table object if needed
            if isinstance(table, str):
                # If 'table' is passed str, use find_table_from_string()
                table = self.find_table_from_string(table)
            # If 'table' is passed as dictionary, we will use that as our table object
            # Get a list of pcsrow objects
            pcsrows = table["pcsRow"]
            # Iterate through each pcsrow
            for pcsrow in pcsrows:
                # Iterate through pos 4,5,6,7 data
                for pos in pcsrow["axis"]:
                    # 'title', 'label', '_pos', '_values'
                    # Narrow search by matching the category text passed as arg
                    if pos["title"] == category:
                        # Extract 'pos' value
                        # NOTE: Can also extract '_values', 'title' key
                        pos_value = pos["_pos"]
                        # Iterate through each 'label' object
                        # Contains '_code', '__text'
                        for code in pos["label"]:
                            # Find the text and code you are looking for
                            if code["__text"] == query_text:
                                code_value = code["_code"]
                                return {
                                    "code": code_value,
                                    "text": query_text,
                                    "pos": pos_value,
                                    "category": category,}
        else:
            # Analyze 'table' agruement to convert to table object if needed
            if isinstance(table, str):
                # If 'table' is passed str, use find_table_from_string()
                table = self.find_table_from_string(table)
            # If 'table' is passed as dictionary, we will use that as our table object
            # Get a list of pcsrow objects
            pcsrows = table["pcsRow"]
            # Iterating through each full pcsrow
            for pcsrow in pcsrows:
                # Iterate through pos 4,5,6,7 data
                for pos in pcsrow["axis"]:
                    # 'title', 'label', '_pos', '_values'
                    # Extract 'category' value
                    category = pos["title"]
                    # Extract 'pos' value
                    # NOTE: Can also extract '_values' key
                    pos_value = pos["_pos"]
                    # Iterating through pos 4,5,6,7 data to find 'text' that matches query_text arg
                    # Iterating through codes available for use in this position
                    for code in pos["label"]:
                        # Check to see if found match to text passed as agruement 
                        if code["__text"] == query_text:
                            return {
                                "code": code["_code"],
                                "text": query_text,
                                "pos": pos["_pos"],
                                "category": category,}

### Notes

* "axis" and "pcsRow" keys contain lists of code, text and definitions

* Table "axis" keys return two key structures
    * {"title", "label", "_pos", "_values"}
    * {"title", "label", "definition", "_pos", "_values"}
* "pcsRow" keys return _ key structures
    * 

In [25]:
# Return table object
table_query_tool = PCS_Table()
test_table = table_query_tool.find_table_from_string("3E0")

In [26]:
# query_tool.search_pcs_rows(test_table, "Approach", "Open")
test = table_query_tool.search_pcs_row_for_components(test_table, "Hormone")

In [27]:
pprint(test)

{'category': 'Substance', 'code': 'V', 'pos': '6', 'text': 'Hormone'}


In [53]:
# return all sections (pos 1 codes)
# Using set to avoid duplicates
pos_1_codes = {(table["axis"][0]["label"][0]["_code"], table["axis"][0]["label"][0]["__text"]) for table in tables}
pos_1_codes = sorted(list(pos_1_codes))
pprint(pos_1_codes)
# Ask user to choose what they want to move forward
# user_input = input("What section do you need? :")

[('0', 'Medical and Surgical'),
 ('1', 'Obstetrics'),
 ('2', 'Placement'),
 ('3', 'Administration'),
 ('4', 'Measurement and Monitoring'),
 ('5', 'Extracorporeal or Systemic Assistance and Performance'),
 ('6', 'Extracorporeal or Systemic Therapies'),
 ('7', 'Osteopathic'),
 ('8', 'Other Procedures'),
 ('9', 'Chiropractic'),
 ('B', 'Imaging'),
 ('C', 'Nuclear Medicine'),
 ('D', 'Radiation Therapy'),
 ('F', 'Physical Rehabilitation and Diagnostic Audiology'),
 ('G', 'Mental Health'),
 ('H', 'Substance Abuse Treatment'),
 ('X', 'New Technology')]


In [66]:
# Hard coded for testing
user_input = "Medical and Surgical"

# Find code that goes with user choice of pos 1 code
for tup in pos_1_codes:
    if tup[1] == user_input:
        user_pos_1_code = tup[0]
        break
        
# Re-query data to find all available pos 2 codes/text
pos_2_codes = {(f"{user_pos_1_code}{table['axis'][1]['label'][0]['_code']}", table["axis"][1]["label"][0]["__text"])
                for table in tables
                if table["axis"][0]["label"][0]["_code"] == user_pos_1_code}
pos_2_codes = sorted(list(pos_2_codes))
pprint(pos_2_codes)
# Ask user to choose what they want to move forward
# user_input = input("What section do you need? :")

[('00', 'Central Nervous System and Cranial Nerves'),
 ('01', 'Peripheral Nervous System'),
 ('02', 'Heart and Great Vessels'),
 ('03', 'Upper Arteries'),
 ('04', 'Lower Arteries'),
 ('05', 'Upper Veins'),
 ('06', 'Lower Veins'),
 ('07', 'Lymphatic and Hemic Systems'),
 ('08', 'Eye'),
 ('09', 'Ear, Nose, Sinus'),
 ('0B', 'Respiratory System'),
 ('0C', 'Mouth and Throat'),
 ('0D', 'Gastrointestinal System'),
 ('0F', 'Hepatobiliary System and Pancreas'),
 ('0G', 'Endocrine System'),
 ('0H', 'Skin and Breast'),
 ('0J', 'Subcutaneous Tissue and Fascia'),
 ('0K', 'Muscles'),
 ('0L', 'Tendons'),
 ('0M', 'Bursae and Ligaments'),
 ('0N', 'Head and Facial Bones'),
 ('0P', 'Upper Bones'),
 ('0Q', 'Lower Bones'),
 ('0R', 'Upper Joints'),
 ('0S', 'Lower Joints'),
 ('0T', 'Urinary System'),
 ('0U', 'Female Reproductive System'),
 ('0V', 'Male Reproductive System'),
 ('0W', 'Anatomical Regions, General'),
 ('0X', 'Anatomical Regions, Upper Extremities'),
 ('0Y', 'Anatomical Regions, Lower Extremitie

* Pass any set of codes and text returned by an index query to 'go-to' table execution function
* Return object where they need to be in table
